In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import os
import fitz
import time
import re
import json
from google.colab import drive
import shutil

### Scrap PDF Links

In [ ]:
# Base URLs
base_url = "https://manshurat.org/taxonomy/term/23?page="
root_url = "https://manshurat.org"

# Output CSV setup
output_file = "extracted_data.csv"
columns = ["Title", "PDF Link", "Sector", "Document Type", "Document Number", "Year"]

In [ ]:
# Open CSV file for writing
with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(columns)

    # Outer loop: Iterate through pages
    for page in range(1, 200):  # 1 to 656
        print(f"Processing page {page}...")
        url = f"{base_url}{page}"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch page {page}")
            continue

        # Parse the main page
        soup = BeautifulSoup(response.content, "html.parser")
        links = soup.select("h2 a")  # All links in <h2> tags

        # Inner loop: Visit each link
        for link in links:
            relative_url = link["href"]
            document_url = f"{root_url}{relative_url}"
            print(f"Fetching document: {document_url}")

            doc_response = requests.get(document_url)
            if doc_response.status_code != 200:
                print(f"Failed to fetch document: {document_url}")
                continue

            # Parse the document page
            doc_soup = BeautifulSoup(doc_response.content, "html.parser")
            try:
                # Extract the six pieces of info
                title = doc_soup.find("div", class_="field-name-title").get_text(strip=True)
                pdf_link = doc_soup.find("div", class_="download-link").find("a")["href"]
                pdf_link = f"{root_url}{pdf_link}"
                inline_info = doc_soup.find_all("div", class_="inline-info")

                sector = inline_info[0].get_text(strip=True).replace("القطاع: ", "")
                document_type = inline_info[1].get_text(strip=True).replace("نوع الوثيقة: ", "")
                document_number = inline_info[2].get_text(strip=True).replace("رقم الوثيقة/رقم الدعوى: ", "")
                year = inline_info[3].get_text(strip=True).replace("سنة الإصدار/السنة القضائية: ", "")

                # Write to CSV
                writer.writerow([title, pdf_link, sector, document_type, document_number, year])

            except Exception as e:
                print(f"Error processing document at {document_url}: {e}")


Processing page 1...
Fetching document: https://manshurat.org/content/tdyl-bd-hkm-qnwn-jyz-ldwl-llmbd-lsgyr-blqnwn-rqm-183-lsn-2023
Fetching document: https://manshurat.org/content/tdyl-bd-hkm-qnwn-lqwbt-blqnwn-rqm-185-lsn-2023
Fetching document: https://manshurat.org/content/tdyl-bd-hkm-qnwn-tnmy-lmshrwt-lmtwst-wlsgyr-wmtnhy-lsgr-blqnwn-rqm-184-lsn-2023
Fetching document: https://manshurat.org/content/sdr-qnwn-d-tnzym-lmjls-lqwm-lltfwl-wlmwm-wtdyl-bd-hkm-qnwn-ltfl
Fetching document: https://manshurat.org/content/sdr-qnwn-lwkl-lmsry-ldmn-lsdrt-wlstthmr
Fetching document: https://manshurat.org/content/tdyl-bd-hkm-qnwn-ldryb-l-lqym-lmdf-blqnwn-177-lsn-2023
Fetching document: https://manshurat.org/content/tdyl-bd-hkm-qnwn-ldryb-l-ldkhl-blqnwn-rqm-175-lsn-2023
Fetching document: https://manshurat.org/content/stthn-qyd-bd-lshrkt-gyr-lmmlwk-llmsryyn-w-lmmlwk-lhm-bnsb-51-mn-bd-hkm-qnwn-sjl-lmstwrdyn
Fetching document: https://manshurat.org/content/tqryr-bd-ltysyrt-llmsryyn-lmqymyn-blkhrj
Fetc

ConnectionError: HTTPSConnectionPool(host='manshurat.org', port=443): Max retries exceeded with url: /node/28007 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000261BBBAA190>: Failed to resolve 'manshurat.org' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
# Specify the range to continue from
start_page = 143 
end_page = 200  

# Open CSV file in append mode
with open(output_file, mode="a", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)

    # Outer loop: Iterate through the specified range of pages
    for page in range(start_page, end_page + 1):  # Adjust range as needed
        print(f"Processing page {page}...")
        url = f"{base_url}{page}"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch page {page}")
            continue

        # Parse the main page
        soup = BeautifulSoup(response.content, "html.parser")
        links = soup.select("h2 a")  # All links in <h2> tags

        # Inner loop: Visit each link
        for link in links:
            relative_url = link["href"]
            document_url = f"{root_url}{relative_url}"
            print(f"Fetching document: {document_url}")

            doc_response = requests.get(document_url)
            if doc_response.status_code != 200:
                print(f"Failed to fetch document: {document_url}")
                continue

            # Parse the document page
            doc_soup = BeautifulSoup(doc_response.content, "html.parser")
            try:
                # Extract the six pieces of info
                title = doc_soup.find("div", class_="field-name-title").get_text(strip=True)
                pdf_link = doc_soup.find("div", class_="download-link").find("a")["href"]
                pdf_link = f"{root_url}{pdf_link}"
                inline_info = doc_soup.find_all("div", class_="inline-info")

                sector = inline_info[0].get_text(strip=True).replace("القطاع: ", "")
                document_type = inline_info[1].get_text(strip=True).replace("نوع الوثيقة: ", "")
                document_number = inline_info[2].get_text(strip=True).replace("رقم الوثيقة/رقم الدعوى: ", "")
                year = inline_info[3].get_text(strip=True).replace("سنة الإصدار/السنة القضائية: ", "")

                # Write to CSV
                writer.writerow([title, pdf_link, sector, document_type, document_number, year])

            except Exception as e:
                print(f"Error processing document at {document_url}: {e}")


Processing page 143...
Fetching document: https://manshurat.org/node/27988
Fetching document: https://manshurat.org/node/27990
Fetching document: https://manshurat.org/node/27991
Fetching document: https://manshurat.org/node/27993
Fetching document: https://manshurat.org/node/27994
Fetching document: https://manshurat.org/node/27995
Fetching document: https://manshurat.org/node/27996
Fetching document: https://manshurat.org/node/27997
Fetching document: https://manshurat.org/node/27999
Fetching document: https://manshurat.org/node/28000
Fetching document: https://manshurat.org/node/28005
Fetching document: https://manshurat.org/node/28007
Fetching document: https://manshurat.org/node/28009
Fetching document: https://manshurat.org/node/28011
Fetching document: https://manshurat.org/node/28013
Fetching document: https://manshurat.org/node/28015
Fetching document: https://manshurat.org/node/28017
Fetching document: https://manshurat.org/node/28018
Fetching document: https://manshurat.org/

In [ ]:
df = pd.read_csv("extracted_data.csv")
df.head()

,Title,PDF Link,Sector,Document Type,Document Number,Year
0,تعديل بعض أحكام قانون جائزة الدولة للمبدع الص...,https://manshurat.org/file/88643/download?toke...,القطاع:شئون اجتماعية وثقافية›الثقافة,نوع الوثيقة:قوانين ولوائح›قوانين,رقم الوثيقة/الدعوى:183,سنة الإصدار/السنة القضائية:2023
1,تعديل بعض أحكام قانون العقوبات بالقانون رقم 18...,https://manshurat.org/file/88644/download?toke...,القطاع:عدالة جنائية وشئون أمنية›الإجراءات الجن...,نوع الوثيقة:قوانين ولوائح›قوانين,رقم الوثيقة/الدعوى:185,سنة الإصدار/السنة القضائية:2023
2,تعديل بعض أحكام قانون تنمية المشروعات المتوسطة...,https://manshurat.org/file/88645/download?toke...,القطاع:سياسات اقتصادية ومالية›التجارة والاستثم...,نوع الوثيقة:قوانين ولوائح›قوانين,رقم الوثيقة/الدعوى:184,سنة الإصدار/السنة القضائية:2023
3,إصدار قانون إعادة تنظيم المجلس القومى للطفولة ...,https://manshurat.org/file/88642/download?toke...,القطاع:شئون اجتماعية وثقافية›قضايا الطفل,نوع الوثيقة:قوانين ولوائح›قوانين,رقم الوثيقة/الدعوى:182,سنة الإصدار/السنة القضائية:2023
4,إصدار قانون الوكالة المصرية لضمان الصادرات وال...,https://manshurat.org/file/88641/download?toke...,القطاع:سياسات اقتصادية ومالية›التجارة والاستثم...,نوع الوثيقة:قوانين ولوائح›قوانين,رقم الوثيقة/الدعوى:178,سنة الإصدار/السنة القضائية:2023


### Download PDF Links

In [ ]:
# Directory to save downloaded PDFs
output_pdf_dir = "downloaded_pdfs"
os.makedirs(output_pdf_dir, exist_ok=True)

# Load the CSV file containing the PDF links
csv_file = "extracted_data.csv"
df = pd.read_csv(csv_file)

# Column names (adjust based on your CSV structure)
pdf_links_column = "PDF Link"  
pdf_titles_column = "Title"

# Limit the number of PDFs to download
max_downloads = 100

# Counter to track downloads
download_count = 0

# Iterate through the CSV and download PDFs
for index, row in df.iterrows():
    if download_count >= max_downloads:
        break

    pdf_url = row[pdf_links_column]
    title = row[pdf_titles_column]

    # Create a safe filename based on the title
    filename = f"{title[:50].replace('/', '_').replace(' ', '_')}.pdf"
    pdf_path = os.path.join(output_pdf_dir, filename)

    try:
        # Download the PDF if it doesn't exist
        if not os.path.exists(pdf_path):
            response = requests.get(pdf_url, stream=True)
            if response.status_code == 200:
                with open(pdf_path, "wb") as pdf_file:
                    pdf_file.write(response.content)
                print(f"Downloaded: {filename}")
                download_count += 1
            else:
                print(f"Failed to download {pdf_url} - HTTP {response.status_code}")

    except Exception as e:
        print(f"Error downloading {filename}: {e}")

print(f"Downloaded {download_count} PDFs to {output_pdf_dir}.")


Downloaded: تعديل_بعض_أحكام__قانون_جائزة_الدولة_للمبدع_الصغير_.pdf
Downloaded: تعديل_بعض_أحكام_قانون_العقوبات_بالقانون_رقم_185_لس.pdf
Downloaded: تعديل_بعض_أحكام_قانون_تنمية_المشروعات_المتوسطة_وال.pdf
Downloaded: إصدار_قانون_إعادة_تنظيم_المجلس_القومى_للطفولة_والأ.pdf
Downloaded: إصدار_قانون_الوكالة_المصرية_لضمان_الصادرات_والاستث.pdf
Downloaded: تعديل_بعض_أحكام_قانون_الضريبة_على_القيمة_المضافة_ب.pdf
Downloaded: تعديل_بعض_أحكام_قانون_الضريبة_على_الدخل_بالقانون_ر.pdf
Downloaded: استثناء_قيد_بعض_الشركات_غير_المملوكة_للمصريين_أو_ا.pdf
Downloaded: تقرير_بعض_التيسيرات_للمصريين_المقيمين_بالخارج.pdf
Downloaded: إلغاء_بعض_القوانين_الخاصة_بإنشاء_بعض_البنوك.pdf
Downloaded: الترخيص_لوزير_البترول_في_التعاقد_مع_الهيئة_العامة_.pdf
Downloaded: دعم_صندوق_قادرون_باختلاف.pdf
Downloaded: تعديل_بعض_أحكام_قانون_إنشاء_هيئة_المحطات_النووية_ل.pdf
Downloaded: إنشاء_الجهاز_المصرى_للملكية_الفكرية.pdf
Downloaded: منح_التزام_تصميم_وإنشاء_وإدارة_وتشغيل_وصيانة_واعاد.pdf
Downloaded: منح_التزام_تمويل_وتصميم_وإنشاء_وادار

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Path to CSV file in Google Drive (Adjust if necessary)
csv_file = "/content/drive/MyDrive/Colab Notebooks/extracted_data.csv"

# Load the CSV file
df = pd.read_csv(csv_file)

# Define start and end index
start_index = 1
end_index = 100

# Filter only the required rows
df = df.iloc[start_index:end_index]

# Google Drive folder to save PDFs (Update this if needed)
output_pdf_dir = "/content/drive/MyDrive/Colab Notebooks/Egyptian Law PDFs Trial"
os.makedirs(output_pdf_dir, exist_ok=True)

# Column names (adjust based on your CSV structure)
pdf_links_column = "PDF Link"  # Replace with actual column name
pdf_titles_column = "Title"  # Replace with actual column name

# Counter to track downloads
download_count = 0

# Iterate through the filtered CSV and download PDFs
for index, row in df.iterrows():
    pdf_url = row[pdf_links_column]
    title = row[pdf_titles_column]

    # Create a safe filename based on the title
    safe_title = title[:40].replace("/", "").replace(" ", "_").replace("\\", "_").replace("?", "").replace(":", "").replace("*", "").replace('"', "").replace("<", "").replace(">", "").replace("|", "")
    filename = f"{index}_{safe_title}.pdf"  # Append index to ensure uniqueness
    # filename = f"{title[:40].replace('/', '_').replace(' ', '_')}.pdf"
    pdf_path = os.path.join(output_pdf_dir, filename)

    try:
        # Download the PDF if it doesn't exist
        if not os.path.exists(pdf_path):
            response = requests.get(pdf_url, stream=True)
            if response.status_code == 200:
                with open(pdf_path, "wb") as pdf_file:
                    pdf_file.write(response.content)
                download_count +=1
                print(f"{download_count} Downloaded: {filename}")
                # download_count += 1
            else:
                print(f"Failed to download {pdf_url} - HTTP {response.status_code}")

    except Exception as e:
        print(f"Error downloading {filename}: {e}")

print(f"\nDownloaded {download_count} PDFs to {output_pdf_dir}.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1 Downloaded: 1_تعديل_بعض_أحكام_قانون_العقوبات_بالقانون_.pdf
2 Downloaded: 2_تعديل_بعض_أحكام_قانون_تنمية_المشروعات_ال.pdf
3 Downloaded: 3_إصدار_قانون_إعادة_تنظيم_المجلس_القومى_لل.pdf
4 Downloaded: 4_إصدار_قانون_الوكالة_المصرية_لضمان_الصادر.pdf
5 Downloaded: 5_تعديل_بعض_أحكام_قانون_الضريبة_على_القيمة.pdf
6 Downloaded: 6_تعديل_بعض_أحكام_قانون_الضريبة_على_الدخل_.pdf
7 Downloaded: 7_استثناء_قيد_بعض_الشركات_غير_المملوكة_للم.pdf
8 Downloaded: 8_تقرير_بعض_التيسيرات_للمصريين_المقيمين_با.pdf
9 Downloaded: 9_إلغاء_بعض_القوانين_الخاصة_بإنشاء_بعض_الب.pdf
10 Downloaded: 10_الترخيص_لوزير_البترول_في_التعاقد_مع_الهي.pdf
11 Downloaded: 11_الترخيص_لوزير_البترول_في_التعاقد_مع_الهي.pdf
12 Downloaded: 12_دعم_صندوق_قادرون_باختلاف.pdf
13 Downloaded: 13_تعديل_بعض_أحكام_قانون_إنشاء_هيئة_المحطات.pdf
14 Downloaded: 14_إنشاء_الجهاز_المصرى_للملكية_الفكرية.pdf
15 Downloaded: 15_منح_التزام

In [ ]:
import os
import requests
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path to CSV file in Google Drive (Adjust if necessary)
csv_file = "/content/drive/MyDrive/Colab Notebooks/extracted_data.csv"

df = pd.read_csv(csv_file)

start_index = 101
end_index = 1700

# Filter only the required rows
df = df.iloc[start_index:end_index]

# Google Drive folder to save PDFs (Update this if needed)
output_pdf_dir = "/content/drive/MyDrive/Colab Notebooks/Egyptian Law 1700 PDFs"
os.makedirs(output_pdf_dir, exist_ok=True)

# Column names
pdf_links_column = "PDF Link"  
pdf_titles_column = "Title"  

# Counter to track downloads
download_count = 0

# Iterate through the filtered CSV and download PDFs
for index, row in df.iterrows():
    pdf_url = row[pdf_links_column]
    title = row[pdf_titles_column]

    # Create a safe filename based on the title
    filename = f"{title[:50].replace('/', '_').replace(' ', '_')}.pdf"
    pdf_path = os.path.join(output_pdf_dir, filename)

    try:
        # Download the PDF if it doesn't exist
        if not os.path.exists(pdf_path):
            response = requests.get(pdf_url, stream=True)
            if response.status_code == 200:
                with open(pdf_path, "wb") as pdf_file:
                    pdf_file.write(response.content)
                download_count +=1
                print(f"{download_count} Downloaded: {filename}")
                # download_count += 1
            else:
                print(f"Failed to download {pdf_url} - HTTP {response.status_code}")

    except Exception as e:
        print(f"Error downloading {filename}: {e}")

print(f"\nDownloaded {download_count} PDFs to {output_pdf_dir}.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1 Downloaded: ربط_حساب_ختامي_موازنة_هيئة_قناة_السويس_للسنة_المال.pdf
2 Downloaded: ربط_حساب_ختامي_موازنة_الهيئة_العامة_لمواني_البحر_ا.pdf
3 Downloaded: ربـط_حساب_ختامي_موازنة_هيئة_ميناء_دمياط_للسنة_الما.pdf
4 Downloaded: ربط_حساب_ختامي_موازنة_الهيئة_العامة_للموانى_البرية.pdf
5 Downloaded: ربط_حساب_ختامي_موازنة_جهاز_تنظيم_النقل_البرى_الداخ.pdf
6 Downloaded: ربط_حساب_ختامي_موازنة_الهيئة_القومية_للبريد_للسنة_.pdf
7 Downloaded: ربط_حساب_ختامي_موازنة_الجهاز_القومى_لتنظيم_الاتصال.pdf
8 Downloaded: ربط_حساب_ختامي_موازنة_هيئة_تنمية_صناعة_تكنولوجيا_ا.pdf
9 Downloaded: ربط_حساب_ختامي_موازنة_وكالة_الفضاء_المصرية_للسنة_ا.pdf
10 Downloaded: ربط_حساب_ختامي_موازنة_الهيئة_القومية_للأنفاق_للسنة.pdf
11 Downloaded: ربط_حساب_ختامي_موازنة_الهيئة_العامة_للسلع_التمويني.pdf
12 Downloaded: ربط_حساب_ختامي_موازنة_الهيئة_العامة_للتحكيم_واختبا.pdf
13 Downloaded: ربط_حساب_ختامي_موازنة_اله

### Insights

In [ ]:
# Path to the folder containing PDFs
output_pdf_dir = "/content/drive/MyDrive/Colab Notebooks/Egyptian Law PDFs Trial"

# Count the number of files in the folder
num_files = len([f for f in os.listdir(output_pdf_dir) if f.endswith(".pdf")])

print(f"Total PDFs in folder: {num_files}")

Total PDFs in folder: 1700


In [ ]:
import os

# Path to the folder containing PDFs
output_pdf_dir = "/content/drive/MyDrive/Colab Notebooks/Egyptian Law 1-1700 PDFs"

# Calculate total size of all PDF files
total_size = sum(
    os.path.getsize(os.path.join(output_pdf_dir, f))
    for f in os.listdir(output_pdf_dir) if f.endswith(".pdf")
)

# Convert size to MB
total_size_mb = total_size / (1024 * 1024)

print(f"Total folder size: {total_size_mb:.2f} MB")


Total folder size: 817.62 MB


In [ ]:

# Folder to compress
folder_path = "/content/drive/MyDrive/Colab Notebooks/Egyptian Law 1-1700 PDFs"


# Output compressed file (ZIP format)
zip_output = " 1-1700 PDFs_compressed"

# Compress the folder
shutil.make_archive(zip_output, 'zip', folder_path)

# Get original and compressed sizes
original_size = sum(os.path.getsize(os.path.join(folder_path, f))
                    for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)))

compressed_size = os.path.getsize(zip_output + ".zip")

# Convert sizes to MB
original_size_mb = original_size / (1024 * 1024)
compressed_size_mb = compressed_size / (1024 * 1024)

print(f"Original folder size: {original_size_mb:.2f} MB")
print(f"Compressed file size: {compressed_size_mb:.2f} MB")
print(f"Compression Ratio: {(compressed_size / original_size) * 100:.2f}%")


Original folder size: 817.62 MB
Compressed file size: 660.82 MB
Compression Ratio: 80.82%


In [ ]:
# Folder containing PDFs
pdf_folder = "/content/drive/MyDrive/Colab Notebooks/Egyptian Law 1-1700 PDFs"

page_counts = []

# Loop through PDFs
for pdf_file in os.listdir(pdf_folder):
    if pdf_file.endswith(".pdf"):  
        pdf_path = os.path.join(pdf_folder, pdf_file)
        try:
            with fitz.open(pdf_path) as pdf_doc:
                num_pages = len(pdf_doc)  
                page_counts.append(num_pages)
        except Exception as e:
            print(f"Error processing {pdf_file}: {e}")

# Calculate average pages per PDF
if page_counts:
    avg_pages = sum(page_counts) / len(page_counts)
    print(f"Average number of pages per PDF: {avg_pages:.2f}")
    print(f"Total number of pages: {sum(page_counts)}")
else:
    print("No PDFs found or couldn't process files.")


Average number of pages per PDF: 8.63


### OCR

In [ ]:
pdf_folder = "/content/drive/MyDrive/Colab Notebooks/Egyptian Law 1-1700 PDFs" 
output_folder = "/content/drive/MyDrive/Colab Notebooks/Egyptian Law OCR Results"  

API_ENDPOINT = "https://genarabia-di.cognitiveservices.azure.com/"  
API_KEY = "Azure_API_Key" 

os.makedirs(output_folder, exist_ok=True)

def extract_numeric_prefix(filename):
    match = re.match(r"(\d+)", filename)  # Extracts leading number
    if match:
        num = match.group(1)  # Get the matched number
        return (len(num), int(num))  # Sort by length first, then numeric value
    return (float("inf"), filename)  # If no number, place it last

# Get a list of all PDF files and sort them properly
pdf_files = sorted(
    [f for f in os.listdir(pdf_folder) if f.endswith(".pdf")],
    key=extract_numeric_prefix
)[336:338]

for filename in pdf_files:
    file_path = os.path.join(pdf_folder, filename)
    pdf_name = os.path.splitext(filename)[0] 

    print(f"Processing: {filename}")

    # Create a folder for this PDF's results
    pdf_output_folder = os.path.join(output_folder, pdf_name)
    os.makedirs(pdf_output_folder, exist_ok=True)

    with open(file_path, "rb") as file:
        pdf_data = file.read()

    headers = {
        "Ocp-Apim-Subscription-Key": API_KEY,
        "Content-Type": "application/pdf"
    }

    # Send the PDF to Azure OCR API
    response = requests.post(
        f"{API_ENDPOINT}/formrecognizer/documentModels/prebuilt-read:analyze?api-version=2023-07-31",
        headers=headers,
        data=pdf_data
    )

    if response.status_code == 202:
        result_url = response.headers.get("Operation-Location")  # Get result URL
        print(f"{filename} uploaded successfully. Processing...")

        # Poll the result URL until processing is done
        while True:
            result_response = requests.get(result_url, headers={"Ocp-Apim-Subscription-Key": API_KEY})
            result_json = result_response.json()

            if result_json.get("status") == "succeeded":
                print(f"{filename} OCR completed")
                break
            elif result_json.get("status") == "failed":
                print(f"{filename} OCR failed")
                break
            else:
                time.sleep(5)  # Wait and retry

        # Save the JSON result
        json_output_path = os.path.join(pdf_output_folder, f"{pdf_name}.json")
        with open(json_output_path, "w", encoding="utf-8") as json_file:
            json.dump(result_json, json_file, ensure_ascii=False, indent=4)

        # Extract text and save as TXT file
        extracted_text = "\n".join(
            line["content"] for page in result_json["analyzeResult"]["pages"] for line in page.get("lines", [])
        )

        text_output_path = os.path.join(pdf_output_folder, f"{pdf_name}.txt")
        with open(text_output_path, "w", encoding="utf-8") as text_file:
            text_file.write(extracted_text)

        print(f"Saved OCR results in: {pdf_output_folder}")

    else:
        print(f"Error processing {filename}: {response.text}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing: 337_فتح_اعتماد_إضافى_بالموازنة_العامة_للدولة.pdf
337_فتح_اعتماد_إضافى_بالموازنة_العامة_للدولة.pdf uploaded successfully. Processing...
337_فتح_اعتماد_إضافى_بالموازنة_العامة_للدولة.pdf OCR completed
Saved OCR results in: /content/drive/MyDrive/Colab Notebooks/Egyptian Law OCR Results/337_فتح_اعتماد_إضافى_بالموازنة_العامة_للدولة
Processing: 338_إرجاء_العمل_بالقانون_رقم_186_لسنة_2020_ب.pdf
338_إرجاء_العمل_بالقانون_رقم_186_لسنة_2020_ب.pdf uploaded successfully. Processing...
338_إرجاء_العمل_بالقانون_رقم_186_لسنة_2020_ب.pdf OCR completed
Saved OCR results in: /content/drive/MyDrive/Colab Notebooks/Egyptian Law OCR Results/338_إرجاء_العمل_بالقانون_رقم_186_لسنة_2020_ب


In [ ]:
base_folder = "/content/drive/MyDrive/Colab Notebooks/Egyptian Law OCR Results"
folder_count = sum(1 for item in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, item)))
print(f"Total number of folders: {folder_count}")

Total number of folders: 1700


### Parsing 

In [ ]:
# Specify the source and destination folders
source_folder = "/content/drive/MyDrive/Colab Notebooks/Renamed Egyptian Laws 1-1700 CSVs"
destination_folder = "/content/drive/MyDrive/Colab Notebooks/Final Renamed Egyptian Laws 1-1700 CSVs"

# Ensure the destination folder exists, create it if it doesn't
os.makedirs(destination_folder, exist_ok=True)

# Copy all files from the source to the destination folder
for filename in os.listdir(source_folder):
    source_file = os.path.join(source_folder, filename)

    # Ensure the destination file name is unique
    destination_file = os.path.join(destination_folder, filename)
    base, extension = os.path.splitext(filename)
    counter = 1

    while os.path.exists(destination_file):
        # Append a number to the filename if it already exists
        destination_file = os.path.join(destination_folder, f"{base}({counter}){extension}")
        counter += 1

    # Copy the file to the destination
    if os.path.isfile(source_file):
        shutil.copy(source_file, destination_file)


In [ ]:
def parse_text_to_csv(input_folder, output_folder,
                      skipped_files_path="/content/drive/MyDrive/Colab Notebooks/skipped_files.txt",
                      article_pattern_str=r"(?<!ال)مادة\s+[\u0660-\u0669]{1,3}",
                      start_marker_str=r"(?<!ال)مادة\s+١",
                      stop_marker_str=r"[،]?[\s]*نشر\s+هذا\s+(?:القرار|القانون)\s+بقانون\s+فى\s+الجريدة\s+الرسمية[\s\u064B-\u0652]*"):

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Load skipped file names if provided
    skipped_files = set()
    if skipped_files_path:
        with open(skipped_files_path, "r", encoding="utf-8") as file:
            skipped_files = {line.strip() for line in file}

    # Compile regex patterns
    article_pattern = re.compile(article_pattern_str)
    start_marker_regex = re.compile(start_marker_str)
    stop_marker = re.compile(stop_marker_str)

    for filename in os.listdir(input_folder):
        if filename.endswith(".txt") and (not skipped_files_path or filename in skipped_files):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename.replace(".txt", ".csv"))

            with open(input_path, "r", encoding="utf-8") as file:
                text = file.read()

            start_match = start_marker_regex.search(text)
            if not start_match:
                print(f"Skipping {filename} due to missing start marker.")
                continue

            text = text[start_match.start():]

            data = []
            current_pos = 0
            seen_articles = set()

            while current_pos < len(text):
                article_match = article_pattern.search(text, current_pos)
                if not article_match:
                    break

                article_full = article_match.group().strip()

                if article_full in seen_articles:
                    current_pos = article_match.end()
                    continue
                seen_articles.add(article_full)

                next_article_match = article_pattern.search(text, article_match.end())
                article_text = text[article_match.end():next_article_match.start() if next_article_match else None].strip()

                if stop_marker.search(article_text):
                    break

                data.append([article_full, article_text])
                current_pos = article_match.end()

            with open(output_path, "w", newline="", encoding="utf-8") as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(["Article Name", "Law Text"])
                writer.writerows(data)

            print(f"Processed: {filename}")

def count_txt_files(folder_path):
    txt_files = [file for file in os.listdir(folder_path) if file.endswith('.txt')]
    count = len(txt_files)
    print(f"Total .txt files in '{folder_path}': {count}")
    return count

In [ ]:
def extract_skipped_files(log_path, output_path='skipped_files.txt'):
    skipped_files = []

    with open(log_path, 'r', encoding='utf-8') as infile:
        for line in infile:
            if line.startswith("Skipping") and "due to missing start marker" in line:
                filename = line.replace("Skipping ", "").replace(" due to missing start marker.", "").strip()
                skipped_files.append(filename)

    with open(output_path, 'w', encoding='utf-8') as outfile:
        for filename in skipped_files:
            outfile.write(filename + '\n')

    print(f"Found {len(skipped_files)} skipped files.")
    print(f"Skipped filenames written to: {output_path}")


extract_skipped_files(
    '/content/drive/MyDrive/Colab Notebooks/processed and skipped files.txt',
    '/content/drive/MyDrive/Colab Notebooks/skipped_files.txt'
)


Found 8 skipped files.
Skipped filenames written to: /content/drive/MyDrive/Colab Notebooks/skipped_files.txt


### Format 1 — "المادة الأولى"

In [ ]:
parse_text_to_csv(
    input_folder="/content/drive/MyDrive/Colab Notebooks/Renamed 1-1700 Text Files",
    output_folder="/content/drive/MyDrive/Colab Notebooks/Renamed Egyptian Laws 1-1700 CSVs",
    skipped_files_path=None,
    article_pattern_str=r"\s*\(?\s*المادة\s+(?!السابقة)([\u0600-\u06FF]+)\s*\)",
    start_marker_str=r"المادة الأولى"
)

Processed: ربط موازنة صندوق تمويل المساكن التي تقيمها وزارة التعمير والمجتمعات الجديدة للسنة المالية 2014  2015.txt
Processed: ربط موازنة الهيئة العامة للتأمين الصحي للسنة المالية 2014  2015.txt
Processed: ربط موازنة هيئة الأوقاف المصرية للسنة المالية 2014  2015.txt
Processed: ربط موازنة المؤسسة العلاجية بالقاهرة للسنة المالية 2014  2015.txt
Processed: ربط موازنة المؤسسة العلاجية لمحافظة الإسكندرية للسنة المالية 2014  2015.txt
Processed: ربط موازنة المؤسسة العلاجية لمحافظة القليوبية للسنة المالية 20142015.txt
Processed: ربط موازنة المعهد القومي للإدارة للسنة المالية 20142015.txt
Processed: ربط موازنة الهيئة القومية لضمان جودة التعليم والباعتماد للسنة المالية 20142015.txt
Processed: ربط موازنة الهيئة العامة للتنمية السياحية للسنة المالية 20142015.txt
Processed: ربط موازنة جهاز الخدمات العامة بوزارة الدفاع للسنة المالية 2014  2015.txt
Processed: ربط موازنة الهيئة القومية للتأمين الاجتماعي للسنة المالية 20142015.txt
Processed: ربط موازنة الهيئة العامة لبنك ناصر الاجتماعي للس

In [ ]:
count_csv_files(csv_folder)

1632

### Format 2 — "الفصل الأول"

In [ ]:
parse_text_to_csv(
    input_folder="/content/drive/MyDrive/Colab Notebooks/Renamed 1-1700 Text Files",
    output_folder="/content/drive/MyDrive/Colab Notebooks/Renamed Egyptian Laws 1-1700 CSVs",
    article_pattern_str=r"\s*\(?\s*الفصل\s+(?!السابق)([\u0600-\u06FF]+)\s*\)",
    start_marker_str=r"الفصل الأول"
)

Processed: تنظيم الانتخابات الرئاسية.txt
Skipping الموافقة على اتفاقية نقل السجناء المحكوم عليهم بعقوبات سالبة للحرية، بين حكومة جمهورية مصر العربية وروسيا الاتحادية.txt due to missing start marker.
Skipping تعديل بعض أحكام قانون البنك المركزي والجهاز المصرفي والنقد بالقانون 160 لسنة 2012.txt due to missing start marker.
Skipping دعم صندوق قادرون باختلاف.txt due to missing start marker.
Processed: التحالف الوطني للعمل الأهلى التنموي.txt
Skipping إلغاء الإعفاءات من الضرائب والرسوم المقررة لجهات الدولة في الأنشطة الاستثمارية والاقتصادية.txt due to missing start marker.
Skipping إنشاء المجلس الأعلى لصناعة السيارات وصندوق تمويل صناعة السيارات صديقة البيئة.txt due to missing start marker.
Skipping منح بعض التيسيرات للمصريين المقيمين بالخارج بالقانون 161 لسنة 2022.txt due to missing start marker.
Processed: إنشاء المركز القومي لإدارة المجال الجوى.txt
Skipping إجراءات مواجهة الأوبئة والجوائح الصحية.txt due to missing start marker.
Skipping إنشاء صندوق الوقف الخيري.txt due to 

In [ ]:
count_csv_files(csv_folder)

1640

### Format 3 — مادة (١): with parentheses and Arabic numerals

In [ ]:
parse_text_to_csv(
    input_folder="/content/drive/MyDrive/Colab Notebooks/Renamed 1-1700 Text Files",
    output_folder="/content/drive/MyDrive/Colab Notebooks/Renamed Egyptian Laws 1-1700 CSVs",
    article_pattern_str=r"(?<!ال)مادة\s*\(\s*[\u0660-\u0669]{1,3}\s*\):?",
    start_marker_str=r"(?<!ال)مادة\s*\(\s*١\s*\):?"
)

Skipping الموافقة على اتفاقية نقل السجناء المحكوم عليهم بعقوبات سالبة للحرية، بين حكومة جمهورية مصر العربية وروسيا الاتحادية.txt due to missing start marker.
Skipping تعديل بعض أحكام قانون البنك المركزي والجهاز المصرفي والنقد بالقانون 160 لسنة 2012.txt due to missing start marker.
Processed: دعم صندوق قادرون باختلاف.txt
Processed: إلغاء الإعفاءات من الضرائب والرسوم المقررة لجهات الدولة في الأنشطة الاستثمارية والاقتصادية.txt
Processed: إنشاء المجلس الأعلى لصناعة السيارات وصندوق تمويل صناعة السيارات صديقة البيئة.txt
Processed: منح بعض التيسيرات للمصريين المقيمين بالخارج بالقانون 161 لسنة 2022.txt
Processed: إجراءات مواجهة الأوبئة والجوائح الصحية.txt
Processed: إنشاء صندوق الوقف الخيري.txt
Skipping إنشاء صندوق مواجهة الطوارئ الطبية.txt due to missing start marker.
Skipping إنشاء صندوق مواجهة الطوارىء الطبية.txt due to missing start marker.
Processed: إنشاء البوابة المصرية للعمرة وتنفيذ الشركات السياحية رحلات العمرة.txt
Skipping منح امتياز إنشاء وتشغيل وإعادة تسليم محطة 

In [ ]:
count_csv_files(csv_folder)

1662

### Format 4 — المادة (١): with "المادة" and Arabic numerals

In [ ]:
parse_text_to_csv(
    input_folder="/content/drive/MyDrive/Colab Notebooks/Renamed 1-1700 Text Files",
    output_folder="/content/drive/MyDrive/Colab Notebooks/Renamed Egyptian Laws 1-1700 CSVs",
    article_pattern_str=r"المادة\s*\(\s*[\u0660-\u0669]{1,3}\s*\):?",
    start_marker_str=r"المادة\s*\(\s*١\s*\):?"
)

Processed: الموافقة على اتفاقية نقل السجناء المحكوم عليهم بعقوبات سالبة للحرية، بين حكومة جمهورية مصر العربية وروسيا الاتحادية.txt
Skipping تعديل بعض أحكام قانون البنك المركزي والجهاز المصرفي والنقد بالقانون 160 لسنة 2012.txt due to missing start marker.
Skipping إنشاء صندوق مواجهة الطوارئ الطبية.txt due to missing start marker.
Skipping إنشاء صندوق مواجهة الطوارىء الطبية.txt due to missing start marker.
Skipping منح امتياز إنشاء وتشغيل وإعادة تسليم محطة دحرجة  رورو  بنظام الـ B. O. T فى ميناء شرق بورسعيد.txt due to missing start marker.
Skipping استدراك للقانون رقم 47 لسنة 2016 بربط موازنة المؤسسة العلاجية لمحافظة القاهرة للسنة المالية 2016  2017.txt due to missing start marker.
Processed: إنشاء بعض المحاكم الابتدائية وتعديل مسمى واختصاص بعض المحاكم الابتدائية الأخرى.txt
Skipping إنشاء مركز الزراعات التعاقدية.txt due to missing start marker.
Skipping قانون رقم 8 لسنة 2015 بتنظيم قوائم الكيانات الإرهابية والإرهابيين.txt due to missing start marker.
Skipping قانون تفضيل ا

In [ ]:
count_csv_files(csv_folder)

1665

### Format 5 — مادة ١ without parentheses

In [ ]:
parse_text_to_csv(
    input_folder="/content/drive/MyDrive/Colab Notebooks/Renamed 1-1700 Text Files",
    output_folder="/content/drive/MyDrive/Colab Notebooks/Renamed Egyptian Laws 1-1700 CSVs",
    article_pattern_str=r"(?<!ال)مادة\s+[\u0660-\u0669]{1,3}",
    start_marker_str=r"(?<!ال)مادة\s+١"
)

Skipping تعديل بعض أحكام قانون البنك المركزي والجهاز المصرفي والنقد بالقانون 160 لسنة 2012.txt due to missing start marker.
Processed: إنشاء صندوق مواجهة الطوارئ الطبية.txt
Processed: إنشاء صندوق مواجهة الطوارىء الطبية.txt
Processed: منح امتياز إنشاء وتشغيل وإعادة تسليم محطة دحرجة  رورو  بنظام الـ B. O. T فى ميناء شرق بورسعيد.txt
Skipping استدراك للقانون رقم 47 لسنة 2016 بربط موازنة المؤسسة العلاجية لمحافظة القاهرة للسنة المالية 2016  2017.txt due to missing start marker.
Processed: إنشاء مركز الزراعات التعاقدية.txt
Processed: قانون رقم 8 لسنة 2015 بتنظيم قوائم الكيانات الإرهابية والإرهابيين.txt
Processed: قانون تفضيل المنتجات المصرية في العقود الحكومية رقم 5 لسنة 2015.txt


In [ ]:
count_csv_files("/content/drive/MyDrive/Colab Notebooks/Renamed Egyptian Laws 1-1700 CSVs")

1671

### Chunking

In [ ]:
folder_path = '/content/drive/MyDrive/New Laws All CSVs'

In [ ]:
folder_path = '/content/drive/MyDrive/New Laws 1-1700 CSVs' 

# Initialize an empty list to store all articles
all_articles = []

# Iterate over all the files in the folder
file_count = 0
article_count = 0
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_count += 1
        law_name = filename.replace('.csv', '')  

        print(f"Processing file: {filename} (Law Name: {law_name})")

        # Open the CSV file
        with open(os.path.join(folder_path, filename), mode='r', encoding='utf-8') as file:
            reader = csv.reader(file)
            # Skip the header row if present
            next(reader, None)

            # Iterate over each row in the CSV file
            for row in reader:
                # Ensure the row contains at least two columns (Article Number, Law Text)
                if len(row) >= 2:
                    article_number = row[0]  # Article Number (first column)
                    law_text = row[1]  # Law Text (second column)

                    # Create a dictionary for each article
                    article = {
                        "Law_Name": law_name,
                        "Article_Number": article_number,
                        "Law_Text": law_text
                    }

                    # Add the article to the list
                    all_articles.append(article)
                    article_count += 1

        print(f"Processed {article_count} articles from {file_count} files so far...")

json_filename = 'articles.json'
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(all_articles, json_file, ensure_ascii=False, indent=4)

print(f"Processing completed! {article_count} articles were processed from {file_count} files.")
print(f"Articles have been saved successfully in '{json_filename}'.")


Streaming output truncated to the last 5000 lines.
Processing file: (8)الترخيص لوزير البترول فى التعاقد مع الهيئة المصرية العامة للبترول وجهات أخرى للبحث عن البترول بخليج السويس.csv (Law Name: (8)الترخيص لوزير البترول فى التعاقد مع الهيئة المصرية العامة للبترول وجهات أخرى للبحث عن البترول بخليج السويس)
Processed 10925 articles from 954 files so far...
Processing file:  الترخيص لوزير البترول في التعاقد للبحث عن البترول واستغلاله في منطقة شرق المرجان بخليج السويس.csv (Law Name:  الترخيص لوزير البترول في التعاقد للبحث عن البترول واستغلاله في منطقة شرق المرجان بخليج السويس)
Processed 11112 articles from 955 files so far...
Processing file: ربط موازنة الهيئة العامة للتحكيم واختبارات القطن للسنة المالية 2003   2004.csv (Law Name: ربط موازنة الهيئة العامة للتحكيم واختبارات القطن للسنة المالية 2003   2004)
Processed 11122 articles from 956 files so far...
Processing file: ربط موازنة جهاز تنظيم مرفق الكهرباء وحماية المستهلك للسنة المالية 2003   2004.csv (Law Name: ربط موازنة جهاز تنظيم مر